In [5]:
import base64

SENSOR_NAMES = {
    0: "Voltaje",
    1: "Corriente",
    2: "Sensor externo 1",
    3: "Sensor externo 2",
    4: "Sensor externo 3",
    5: "Sensor externo 4",
    6: "Sensor externo 5",
    7: "Sensor externo 6",
}

def decode_unified_payload(b64):
    data = base64.b64decode(b64)
    pos = 0
    id_msg = data[pos]; pos += 1
    timestamp = int.from_bytes(data[pos:pos+4], "big"); pos += 4
    activate = data[pos]; pos += 1
    bat_raw = data[pos]; pos += 1

    # read len descriptors
    len_desc = []
    for bit in range(8):
        if (activate >> bit) & 1:
            lb = data[pos]; pos += 1
            len_desc.append((bit, lb))

    parsed = {
        "ID del mensaje": id_msg,
        "Timestamp": timestamp,
        "Batería (V)": bat_raw / 10.0,
        "Sensores activos": [SENSOR_NAMES[b] for b in range(8) if (activate >> b) & 1],
        "Datos": {}
    }

    # Voltaje (3 canales, un byte por muestra)
    if activate & 0x01:
        _, lb = len_desc.pop(0)
        length = lb & 0x1F
        volts = []
        for ch in range(3):
            chvals = list(data[pos:pos+length])
            pos += length
            volts.append(chvals)
        parsed["Datos"]["Voltaje"] = {
            "Canal 1": volts[0],
            "Canal 2": volts[1],
            "Canal 3": volts[2],
        }

    # Corriente (3 canales, 10 bits empaquetados)
    if activate & 0x02:
        _, lb = len_desc.pop(0)
        length = lb & 0x1F
        total_10vals = 3 * length
        total_bits = total_10vals * 10
        total_bytes = (total_bits + 7) // 8
        raw = data[pos:pos+total_bytes]; pos += total_bytes
        bitstr = "".join(f"{b:08b}" for b in raw)
        vals10 = [int(bitstr[i*10:(i+1)*10].ljust(10, "0"), 2) for i in range(total_10vals)]
        ch1 = [v/10.0 for v in vals10[0:length]]
        ch2 = [v/10.0 for v in vals10[length:2*length]]
        ch3 = [v/10.0 for v in vals10[2*length:3*length]]
        parsed["Datos"]["Corriente"] = {
            "Canal 1": ch1,
            "Canal 2": ch2,
            "Canal 3": ch3,
        }

    # Sensores externos (bytes crudos)
    for bit in range(2,8):
        if (activate >> bit) & 1:
            _, lb = len_desc.pop(0)
            length = lb & 0x1F
            parsed["Datos"][SENSOR_NAMES[bit]] = list(data[pos:pos+length])
            pos += length

    return parsed


# Ejemplo de uso:
payload_b64 = "4AAABTsDABSUfn5+fn5+fn5+fn5+fn5+fn5+fn6AgICAgICAgICAgICAgICAgICAgHp7e3t7e3t7e3p6enp6enp6enp7AQBQEAMAgCAIAgBAEAQBAEAQBAEAQBAEAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
res = decode_unified_payload(payload_b64)

# Mostrar resultados bonitos
print("📡 Decodificación del payload")
print("ID del mensaje:", res["ID del mensaje"])
print("Timestamp:", res["Timestamp"])
print("Batería:", res["Batería (V)"], "V")
print("Sensores activos:", ", ".join(res["Sensores activos"]))
print("\n--- Datos de sensores ---")
for sensor, valores in res["Datos"].items():
    print(f"\n🔹 {sensor}:")
    if isinstance(valores, dict):
        for canal, datos in valores.items():
            print(f"  {canal}: {datos}")
    else:
        print(" ", valores)


📡 Decodificación del payload
ID del mensaje: 224
Timestamp: 1339
Batería: 0.0 V
Sensores activos: Voltaje, Corriente

--- Datos de sensores ---

🔹 Voltaje:
  Canal 1: [126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126]
  Canal 2: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]
  Canal 3: [122, 123, 123, 123, 123, 123, 123, 123, 123, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 123]

🔹 Corriente:
  Canal 1: [0.4, 0.5, 0.4, 0.3, 0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
  Canal 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Canal 3: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
